In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit",
    # model_name = 'unsloth/llama-3-8b-Instruct-bnb-4bit',
    # model_name = 'unsloth/mistral-7b-instruct-v0.3-bnb-4bit',
    model_name = 'unsloth/mistral-7b-v0.3-bnb-4bit',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # rank of LoRA
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none", # No bias added for optimal performance
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import pandas as pd
from datasets import Dataset

In [14]:
EOS_TOKEN = tokenizer.eos_token
df = pd.read_excel('train.xlsx')
# prompt = "Summarize the provided code solution for the given problem in simple, plain English text. Explain in simple text how the code works to solve the specified problem."
prompt = "Summarize the provided code solution for the given problem in simple, plain English. Explain step-by-step how the code works to solve the problem in a way that is easy to understand."
df["question"] = (
    prompt
    + "\n Question: )"
    + df["Question"]
    + "\n Code: )"
    + df["Code"]
    + "\n Plain Text: )"
    + df["Plain Text"]
    + EOS_TOKEN
)

custom_ds = pd.DataFrame()
custom_ds["text"] = df["question"]
dataset = Dataset.from_pandas(custom_ds)

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 15,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/20 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [16]:
trainer_stats = trainer.train()

Step,Training Loss
1,1.017700
2,0.568200
3,0.517900
4,0.983000
5,0.853500
6,0.762200
7,0.748900
8,0.648600
9,1.047200
10,0.455300


In [17]:
test_df = pd.read_excel('test.xlsx')
test_row = test_df.tail(1)
test_text = prompt + "\n Question: )" + test_row["Question"].values[0] + "\n Code: )" + test_row["Code"].values[0] + "\n Plain Text: )"
print(test_text)

Summarize the provided code solution for the given problem in simple, plain English. Explain step-by-step how the code works to solve the problem in a way that is easy to understand.
 Question: )Given a string s containing just the characters '(', ')', '{', '}', '[' and ']', determine if the input string is valid.

An input string is valid if:

Open brackets must be closed by the same type of brackets.
Open brackets must be closed in the correct order.
Every close bracket has a corresponding open bracket of the same type.
 

Example 1:

Input: s = "()"
Output: true
Example 2:

Input: s = "()[]{}"
Output: true
 Code: )class Solution:
    def isValid(self, s: str) -> bool:
        Map = {")": "(", "]": "[", "}": "{"}
        stack = []

        for c in s:
            if c not in Map:
                stack.append(c)
                continue
            if not stack or stack[-1] != Map[c]:
                return False
            stack.pop()

        return not stack

 Plain Text: )


In [18]:
FastLanguageModel.for_inference(model)

inputs = tokenizer(
    [test_text],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=200)

# Decode the generated tokens, excluding the input tokens
generated_sequence = outputs[0]
input_length = inputs['input_ids'].shape[1]  # Length of the input text tokens
new_tokens = generated_sequence[input_length:]  # Exclude the input tokens

new_text = tokenizer.decode(new_tokens, skip_special_tokens=True)

print(new_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


To determine if a string containing parentheses, brackets, and curly brackets is valid, use a stack. Iterate through the string, pushing opening characters onto the stack and popping them when their corresponding closing characters are encountered. If the stack is empty at the end, the string is valid; otherwise, it is not.
